## Library Installation
Before running the notebook, ensure that the required libraries are installed. You can install them using the following commands:

# OSM Features Extraction for Air Quality Data
This notebook extracts OpenStreetMap features based on latitude and longitude for air quality data labeling.

## Import Required Libraries
Import libraries such as pandas, requests, and any mapping libraries needed.

In [2]:
# Library Installation
!pip install pandas requests geopy overpy

In [3]:
# Import Required Libraries
import pandas as pd
import requests
import json
from geopy.geocoders import Nominatim
import overpy

# Initialize Overpass API
api = overpy.Overpass()

## Load Station Locations from CSV
Load the station locations from the provided CSV file using pandas.

In [4]:
# Load Station Locations from CSV
df_stations = pd.read_csv('station_locations.csv')
print(df_stations.head())

                                      station_name station_id   latitude  \
0                SIDCO Kurichi, Coimbatore - TNPCB  site_5094  10.942451   
1                    Urban, Chamarajanagar - KSPCB  site_5124  11.553580   
2                    MD University, Rohtak - HSPCB   site_147  28.521230   
3  IESD Banaras Hindu University, Varanasi - UPPCB  site_5468  25.262326   
4                           Sirifort, Delhi - CPCB   site_119  28.550425   

   longitude  city  state  
0  76.978996   NaN    NaN  
1  76.555210   NaN    NaN  
2  76.371380   NaN    NaN  
3  82.995408   NaN    NaN  
4  77.215938   NaN    NaN  


## Define Function to Fetch OSM Features
Create a function that takes latitude and longitude as input and fetches relevant OSM features using an API.

In [5]:
# Define Function to Fetch OSM Features

def fetch_osm_features(lat, lon):
    # Define your Overpass API query here
    query = f'[out:json];(node(around:500,{lat},{lon}););out;'
    result = api.query(query)
    relevant_features = []
    for node in result.nodes:
        features = node.tags
        # Check for relevant features
        if any(feature in features for feature in ['aerialway', 'amenity', 'building', 'highway', 'landuse', 'natural', 'shop']):
            relevant_features.append(features)
    return relevant_features

## Map Latitude and Longitude to OSM Features
Iterate through the station locations and use the function to map each location to its corresponding OSM features.

In [12]:
import requests
import time

# Example: bounding box around the first station (small area, ~500m buffer)
lat = float(df_stations.iloc[0]['latitude'])
lon = float(df_stations.iloc[0]['longitude'])
delta = 0.005  # ~500m in degrees

bbox = f"{lat-delta},{lon-delta},{lat+delta},{lon+delta}"

query = f"""
[bbox:{bbox}]
[out:json]
[timeout:90];
(
  node({lat-delta},{lon-delta},{lat+delta},{lon+delta});
  way({lat-delta},{lon-delta},{lat+delta},{lon+delta});
  relation({lat-delta},{lon-delta},{lat+delta},{lon+delta});
);
out body;
>;
out skel qt;
"""

url = "https://overpass-api.de/api/interpreter"

print(f"Querying Overpass API for station: {df_stations.iloc[0]['station_name']}")
print(f"Location: {lat}, {lon}")
print(f"Bounding box: {bbox}\n")

# Add error handling
try:
    response = requests.post(url, data={'data': query}, timeout=120)
    
    print(f"Response status code: {response.status_code}")
    print(f"Response content type: {response.headers.get('Content-Type', 'Unknown')}")
    
    # Check if response is successful
    if response.status_code == 200:
        # Print first 500 chars of response to debug
        print(f"Response preview (first 500 chars):\n{response.text[:500]}\n")
        
        # Try to parse JSON
        data = response.json()
        
        # Print a summary of the result
        print(f"✓ Successfully received data!")
        print(f"Number of elements: {len(data.get('elements', []))}")
        
        if data.get('elements'):
            print(f"\nFirst 3 elements:")
            for i, elem in enumerate(data['elements'][:3]):
                print(f"\nElement {i+1}:")
                print(f"  Type: {elem.get('type')}")
                print(f"  ID: {elem.get('id')}")
                print(f"  Tags: {elem.get('tags', {})}")
    elif response.status_code == 429:
        print("⚠ Rate limited! Too many requests. Wait a moment and try again.")
    elif response.status_code == 504:
        print("⚠ Gateway timeout! The query took too long. Try reducing the search area.")
    else:
        print(f"⚠ Error: {response.status_code}")
        print(f"Response text:\n{response.text[:1000]}")
        
except requests.exceptions.Timeout:
    print("⚠ Request timed out! The server took too long to respond.")
except requests.exceptions.RequestException as e:
    print(f"⚠ Request failed: {e}")
except json.JSONDecodeError as e:
    print(f"⚠ Failed to parse JSON response: {e}")
    print(f"Response text:\n{response.text[:1000]}")

Querying Overpass API for station: SIDCO Kurichi, Coimbatore - TNPCB
Location: 10.942451, 76.978996
Bounding box: 10.937451,76.973996,10.947451000000001,76.98399599999999

Response status code: 200
Response content type: application/json
Response preview (first 500 chars):
{
  "version": 0.6,
  "generator": "Overpass API 0.7.62.10 2d4cfc48",
  "osm3s": {
    "timestamp_osm_base": "2026-01-29T06:18:10Z",
    "copyright": "The data included in this document is from www.openstreetmap.org. The data is made available under ODbL."
  },
  "elements": [

{
  "type": "node",
  "id": 266585747,
  "lat": 10.9391913,
  "lon": 76.9810669
},
{
  "type": "node",
  "id": 266585748,
  "lat": 10.9392998,
  "lon": 76.9799146
},
{
  "type": "node",
  "id": 1423799063,
  "lat": 10.937

✓ Successfully received data!
Number of elements: 22953

First 3 elements:

Element 1:
  Type: node
  ID: 266585747
  Tags: {}

Element 2:
  Type: node
  ID: 266585748
  Tags: {}

Element 3:
  Type: node
  ID: 1423799063
  T

## Label Mapping for Air Quality Data
Create a mapping of the fetched OSM features to labels relevant for air quality data.

In [13]:
# Extract OSM Feature Labels for ML Training
def extract_osm_feature_labels(osm_data):
    """
    Extract OSM features as LABELS (not counts).
    Each primary feature key becomes a column with comma-separated values.
    
    Example output:
    - highway: "trunk,secondary,residential"
    - landuse: "industrial,commercial"
    - amenity: "fuel,parking,hospital"
    
    Returns:
        dict: Feature labels {key: "value1,value2,value3"}
    """
    from collections import defaultdict
    
    # Store unique values for each feature key
    feature_values = defaultdict(set)
    
    # PRIMARY feature keys from OSM Map Features
    primary_feature_keys = {
        'aerialway', 'aeroway', 'highway', 'railway', 'public_transport',
        'landuse', 'natural', 'leisure', 'place', 'building',
        'amenity', 'shop', 'tourism', 'office',
        'man_made', 'power', 'craft', 'industrial',
        'emergency', 'healthcare',
        'waterway', 'water', 'geological',
        'barrier', 'boundary', 'historic', 'military', 'sport'
    }
    
    # Process OSM elements
    elements = osm_data.get('elements', [])
    total_elements = len(elements)
    
    for element in elements:
        tags = element.get('tags', {})
        
        for key, value in tags.items():
            # ONLY include primary feature keys
            if key not in primary_feature_keys:
                continue
            
            # Skip generic/non-informative values
            skip_values = ['yes', 'no', 'unknown', '']
            if value in skip_values:
                continue
            
            # Clean the value
            safe_value = str(value).replace(' ', '_').replace(':', '_').replace('-', '_').replace(',', '_').replace('/', '_').replace('.', '_')
            
            # Add to set (automatically handles duplicates)
            feature_values[key].add(safe_value)
    
    # Convert sets to comma-separated strings
    feature_labels = {}
    for key, values in feature_values.items():
        # Sort values for consistency
        sorted_values = sorted(list(values))
        feature_labels[key] = ','.join(sorted_values)
    
    # Add metadata
    feature_labels['_total_elements'] = total_elements
    feature_labels['_unique_feature_types'] = len(feature_values)
    
    return feature_labels, feature_values


# Apply feature extraction to the data we retrieved
if 'data' in locals() and data.get('elements'):
    print("Extracting OSM feature labels...\n")
    feature_labels, feature_values_dict = extract_osm_feature_labels(data)
    
    # Display the feature extraction results
    print("=" * 70)
    print("OSM FEATURE LABELS EXTRACTION RESULTS")
    print("=" * 70)
    print(f"Total OSM elements found: {feature_labels.get('_total_elements', 0)}")
    print(f"Unique feature types: {feature_labels.get('_unique_feature_types', 0)}")
    
    print("\n" + "-" * 70)
    print("EXTRACTED FEATURE LABELS BY CATEGORY")
    print("-" * 70)
    
    # Display features by category
    for key in sorted(feature_values_dict.keys()):
        values = feature_labels[key]
        value_list = values.split(',')
        print(f"\n{key.upper()}:")
        print(f"  Labels: {values}")
        print(f"  Count of unique values: {len(value_list)}")
    
    # Create DataFrame for this station
    feature_df = pd.DataFrame([feature_labels])
    feature_df.insert(0, 'station_id', df_stations.iloc[0]['station_id'])
    feature_df.insert(1, 'station_name', df_stations.iloc[0]['station_name'])
    feature_df.insert(2, 'latitude', df_stations.iloc[0]['latitude'])
    feature_df.insert(3, 'longitude', df_stations.iloc[0]['longitude'])
    
    print("\n" + "=" * 70)
    print("FEATURE DATAFRAME CREATED")
    print("=" * 70)
    print(f"Shape: {feature_df.shape}")
    print(f"Columns: {len(feature_df.columns)}")
    print(f"\nColumns: {list(feature_df.columns)}")
    
    # Show sample data
    print("\n" + "-" * 70)
    print("SAMPLE DATA (First Row)")
    print("-" * 70)
    for col in feature_df.columns:
        if not col.startswith('_'):
            print(f"{col:20s}: {feature_df[col].iloc[0]}")
    
    print("\n✓ Feature label extraction complete!")
    print("  Each column contains comma-separated labels for that feature type.")
    print("  This labeled data can be used for air quality prediction!")
    
else:
    print("⚠ No OSM data available. Please run the previous cell to fetch data first.")

Extracting OSM feature labels...

OSM FEATURE LABELS EXTRACTION RESULTS
Total OSM elements found: 22953
Unique feature types: 6

----------------------------------------------------------------------
EXTRACTED FEATURE LABELS BY CATEGORY
----------------------------------------------------------------------

AMENITY:
  Labels: college,fuel
  Count of unique values: 2

BARRIER:
  Labels: gate
  Count of unique values: 1

BOUNDARY:
  Labels: administrative
  Count of unique values: 1

HIGHWAY:
  Labels: residential,service,tertiary,trunk,unclassified
  Count of unique values: 5

LANDUSE:
  Labels: industrial
  Count of unique values: 1

RAILWAY:
  Labels: rail
  Count of unique values: 1

FEATURE DATAFRAME CREATED
Shape: (1, 12)
Columns: 12

Columns: ['station_id', 'station_name', 'latitude', 'longitude', 'barrier', 'highway', 'railway', 'landuse', 'amenity', 'boundary', '_total_elements', '_unique_feature_types']

----------------------------------------------------------------------
SAM

## Save Mapped Features to CSV
Save the mapped features and labels to a new CSV file for further analysis.

In [14]:
# Save Extracted Features to CSV
if 'feature_df' in locals():
    output_file = 'station_osm_features.csv'
    feature_df.to_csv(output_file, index=False)
    print(f"✓ Feature data saved to '{output_file}'")
    print(f"  - Rows: {len(feature_df)}")
    print(f"  - Columns: {len(feature_df.columns)}")
    print(f"\nThis CSV can be used for ML model training to predict air quality based on location features!")
else:
    print("⚠ No feature data to save. Please run the feature extraction cell first.")

✓ Feature data saved to 'station_osm_features.csv'
  - Rows: 1
  - Columns: 12

This CSV can be used for ML model training to predict air quality based on location features!


## Batch Process Multiple Stations
Process multiple stations with automatic retry and rate limiting (5 second delay between requests).

In [7]:
# Batch Process Multiple Stations
import time

# ========== CONFIGURATION ==========
NUM_STATIONS = 25  # Change this to process more stations
DELAY_SECONDS = 5  # Delay between requests to avoid rate limiting
RETRY_DELAY = 15  # Delay after a failed request before retrying
# ===================================

print(f"Starting batch processing for {NUM_STATIONS} stations...")
print(f"Delay between requests: {DELAY_SECONDS} seconds")
print("=" * 70)

# Storage for all station features
all_features = []
failed_stations = []

for idx in range(min(NUM_STATIONS, len(df_stations))):
    station = df_stations.iloc[idx]
    station_id = station['station_id']
    station_name = station['station_name']
    lat = float(station['latitude'])
    lon = float(station['longitude'])
    
    print(f"\n[{idx+1}/{NUM_STATIONS}] Processing: {station_name}")
    print(f"  Location: ({lat}, {lon})")
    
    # Build Overpass query
    delta = 0.005
    bbox = f"{lat-delta},{lon-delta},{lat+delta},{lon+delta}"
    
    query = f"""
    [bbox:{bbox}]
    [out:json]
    [timeout:90];
    (
      node({lat-delta},{lon-delta},{lat+delta},{lon+delta});
      way({lat-delta},{lon-delta},{lat+delta},{lon+delta});
      relation({lat-delta},{lon-delta},{lat+delta},{lon+delta});
    );
    out body;
    >;
    out skel qt;
    """
    
    url = "https://overpass-api.de/api/interpreter"
    
    # Try to fetch data with retry logic
    max_retries = 5
    retry_count = 0
    success = False
    
    while retry_count < max_retries and not success:
        try:
            print(f"  Querying Overpass API... (attempt {retry_count+1}/{max_retries})")
            response = requests.post(url, data={'data': query}, timeout=120)
            
            if response.status_code == 200:
                data = response.json()
                elements_count = len(data.get('elements', []))
                print(f"  ✓ Success! Received {elements_count} elements")
                
                # Extract feature labels
                feature_labels, _ = extract_osm_feature_labels(data)
                
                # Add station metadata
                feature_labels['station_id'] = station_id
                feature_labels['station_name'] = station_name
                feature_labels['latitude'] = lat
                feature_labels['longitude'] = lon
                
                all_features.append(feature_labels)
                success = True
                
            elif response.status_code == 429:
                print(f"  ⚠ Rate limited! Waiting {RETRY_DELAY} seconds...")
                time.sleep(RETRY_DELAY)
                retry_count += 1
                
            elif response.status_code == 504:
                print(f"  ⚠ Gateway timeout! Waiting {RETRY_DELAY} seconds...")
                time.sleep(RETRY_DELAY)
                retry_count += 1
                
            else:
                print(f"  ⚠ Error {response.status_code}: {response.text[:200]}")
                retry_count += 1
                
        except requests.exceptions.Timeout:
            print(f"  ⚠ Request timed out! Waiting {RETRY_DELAY} seconds...")
            time.sleep(RETRY_DELAY)
            retry_count += 1
            
        except Exception as e:
            print(f"  ⚠ Error: {e}")
            retry_count += 1
    
    if not success:
        print(f"  ✗ Failed after {max_retries} attempts")
        failed_stations.append({'station_id': station_id, 'station_name': station_name})
    
    # Wait before next request (except for the last one)
    if idx < NUM_STATIONS - 1:
        print(f"  Waiting {DELAY_SECONDS} seconds before next request...")
        time.sleep(DELAY_SECONDS)

# Create final DataFrame
print("\n" + "=" * 70)
print("BATCH PROCESSING COMPLETE")
print("=" * 70)
print(f"Successfully processed: {len(all_features)} stations")
print(f"Failed: {len(failed_stations)} stations")

if all_features:
    # Create DataFrame with all features
    batch_df = pd.DataFrame(all_features)
    
    # Reorder columns: station info first, then features
    info_cols = ['station_id', 'station_name', 'latitude', 'longitude']
    feature_cols = [col for col in batch_df.columns if col not in info_cols]
    batch_df = batch_df[info_cols + sorted(feature_cols)]
    
    print(f"\nDataFrame shape: {batch_df.shape}")
    print(f"Columns: {len(batch_df.columns)}")
    
    # Save to CSV
    output_file = f'station_osm_features_batch_{NUM_STATIONS}.csv'
    batch_df.to_csv(output_file, index=False)
    print(f"\n✓ Saved to '{output_file}'")
    
    # Display summary
    print("\n" + "-" * 70)
    print("SUMMARY OF EXTRACTED FEATURES")
    print("-" * 70)
    print(batch_df.head())
    
else:
    print("\n⚠ No data was successfully extracted!")

if failed_stations:
    print("\n" + "-" * 70)
    print("FAILED STATIONS")
    print("-" * 70)
    for failed in failed_stations:
        print(f"  - {failed['station_name']} ({failed['station_id']})")

Starting batch processing for 25 stations...
Delay between requests: 5 seconds

[1/25] Processing: SIDCO Kurichi, Coimbatore - TNPCB
  Location: (10.942451, 76.978996)
  Querying Overpass API... (attempt 1/5)


KeyboardInterrupt: 

## Unify Station IDs Across Datasets
Map station IDs from different sources to create a unified dataset with consistent station identifiers.

In [16]:
# Load all datasets
print("Loading datasets...")
print("=" * 70)

# Load stations.csv (has state-based IDs like AP001, DL001)
df_stations_master = pd.read_csv('stations.csv')
print(f"✓ Loaded stations.csv: {len(df_stations_master)} stations")
print(f"  Sample IDs: {df_stations_master['StationId'].head(3).tolist()}")

# Load station_locations.csv (has site_XXXX IDs)
df_locations = pd.read_csv('station_locations.csv')
print(f"\n✓ Loaded station_locations.csv: {len(df_locations)} stations")
print(f"  Sample IDs: {df_locations['station_id'].head(3).tolist()}")

# Load OSM features (has site_XXXX IDs)
df_osm = pd.read_csv('station_osm_features_batch_230.csv')
print(f"\n✓ Loaded station_osm_features_batch_230.csv: {len(df_osm)} stations")
print(f"  Sample IDs: {df_osm['station_id'].head(3).tolist()}")

print("\n" + "=" * 70)
print("DATASET SUMMARY")
print("=" * 70)
print(f"Total unique station names in stations.csv: {df_stations_master['StationName'].nunique()}")
print(f"Total unique station names in station_locations.csv: {df_locations['station_name'].nunique()}")
print(f"Total unique station names in OSM features: {df_osm['station_name'].nunique()}")

Loading datasets...
✓ Loaded stations.csv: 230 stations
  Sample IDs: ['AP001', 'AP002', 'AP003']

✓ Loaded station_locations.csv: 567 stations
  Sample IDs: ['site_5094', 'site_5124', 'site_147']

✓ Loaded station_osm_features_batch_230.csv: 230 stations
  Sample IDs: ['site_5094', 'site_5124', 'site_147']

DATASET SUMMARY
Total unique station names in stations.csv: 230
Total unique station names in station_locations.csv: 567
Total unique station names in OSM features: 230


In [19]:
# Create unified station ID mapping based on station names
print("Creating unified station ID mapping...")
print("=" * 70)

# Function to normalize station names for matching
def normalize_name(name):
    """Normalize station name for matching (remove extra spaces, lowercase)"""
    if pd.isna(name):
        return ""
    return str(name).strip().lower()

# Create normalized name columns
df_stations_master['normalized_name'] = df_stations_master['StationName'].apply(normalize_name)
df_locations['normalized_name'] = df_locations['station_name'].apply(normalize_name)
df_osm['normalized_name'] = df_osm['station_name'].apply(normalize_name)

# Create mapping from site_XXXX to state-based IDs (using station names as keys)
name_to_state_id = dict(zip(df_stations_master['normalized_name'], df_stations_master['StationId']))

# Map site_XXXX to state-based IDs for station_locations
print("\nMapping station_locations.csv IDs...")
df_locations['unified_station_id'] = df_locations['normalized_name'].map(name_to_state_id)
matched_locations = df_locations['unified_station_id'].notna().sum()
unmatched_locations = df_locations['unified_station_id'].isna().sum()
print(f"  Matched: {matched_locations}/{len(df_locations)} stations")
print(f"  Unmatched: {unmatched_locations}/{len(df_locations)} stations")

# For unmatched, keep original site_XXXX ID
df_locations['unified_station_id'].fillna(df_locations['station_id'], inplace=True)

# Map site_XXXX to state-based IDs for OSM features
print("\nMapping OSM features IDs...")
df_osm['unified_station_id'] = df_osm['normalized_name'].map(name_to_state_id)
matched_osm = df_osm['unified_station_id'].notna().sum()
unmatched_osm = df_osm['unified_station_id'].isna().sum()
print(f"  Matched: {matched_osm}/{len(df_osm)} stations")
print(f"  Unmatched: {unmatched_osm}/{len(df_osm)} stations")

# For unmatched, keep original site_XXXX ID
df_osm['unified_station_id'].fillna(df_osm['station_id'], inplace=True)

print("\n" + "=" * 70)
print("UNIFIED ID MAPPING COMPLETE")
print("=" * 70)

# Show some examples of the mapping
print("\nExample mappings (first 10 matched):")
print("-" * 70)
mapping_examples = df_locations[df_locations['normalized_name'].isin(name_to_state_id)].head(10)
for _, row in mapping_examples.iterrows():
    print(f"  {row['station_id']:12s} → {row['unified_station_id']:8s}  |  {row['station_name'][:50]}")

# Show unmatched stations
if unmatched_locations > 0:
    print(f"\nUnmatched stations in station_locations.csv (keeping original site_XXXX IDs):")
    print("-" * 70)
    unmatched = df_locations[~df_locations['normalized_name'].isin(name_to_state_id)].head(10)
    for _, row in unmatched.iterrows():
        print(f"  {row['station_id']:12s} (no match)  |  {row['station_name'][:50]}")

Creating unified station ID mapping...

Mapping station_locations.csv IDs...
  Matched: 218/567 stations
  Unmatched: 349/567 stations

Mapping OSM features IDs...
  Matched: 191/230 stations
  Unmatched: 39/230 stations

UNIFIED ID MAPPING COMPLETE

Example mappings (first 10 matched):
----------------------------------------------------------------------
  site_5094    → TN005     |  SIDCO Kurichi, Coimbatore - TNPCB
  site_5124    → KA012     |  Urban, Chamarajanagar - KSPCB
  site_147     → HR026     |  MD University, Rohtak - HSPCB
  site_119     → DL034     |  Sirifort, Delhi - CPCB
  site_1549    → PB002     |  Hardev Nagar, Bathinda - PPCB
  site_5115    → MH014     |  Worli, Mumbai - MPCB
  site_288     → TN004     |  Velachery Res. Area, Chennai - CPCB
  site_5269    → MP013     |  Deen Dayal Nagar, Sagar - MPPCB
  site_5039    → HR021     |  Sector-2 IMT, Manesar - HSPCB
  site_5092    → TN002     |  Manali Village, Chennai - TNPCB

Unmatched stations in station_locations.cs

/var/folders/9_/xsy51vhx14z013rcw2jjgtlm0000gn/T/ipykernel_33855/3142674048.py:29: ChainedAssignmentError: A value is being set on a copy of a DataFrame or Series through chained assignment using an inplace method.
Such inplace method never works to update the original DataFrame or Series, because the intermediate object on which we are setting values always behaves as a copy (due to Copy-on-Write).

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' instead, to perform the operation inplace on the original object, or try to avoid an inplace operation using 'df[col] = df[col].method(value)'.

See the documentation for a more detailed explanation: https://pandas.pydata.org/pandas-docs/stable/user_guide/copy_on_write.html
  df_locations['unified_station_id'].fillna(df_locations['station_id'], inplace=True)
/var/folders/9_/xsy51vhx14z013rcw2jjgtlm0000gn/T/ipykernel_33855/3142674048.py:40: ChainedAssignmentError: A value is bein

In [20]:
# Create unified datasets with consistent station IDs
print("Creating unified datasets...")
print("=" * 70)

# 1. Create unified station_locations with state-based IDs
df_locations_unified = df_locations.copy()
df_locations_unified['original_station_id'] = df_locations_unified['station_id']
df_locations_unified['station_id'] = df_locations_unified['unified_station_id']
df_locations_unified = df_locations_unified.drop(['normalized_name', 'unified_station_id'], axis=1)

# Reorder columns
cols = ['station_id', 'original_station_id', 'station_name', 'latitude', 'longitude', 'city', 'state']
df_locations_unified = df_locations_unified[cols]

print(f"✓ Created unified station_locations: {len(df_locations_unified)} stations")
print(f"  Columns: {list(df_locations_unified.columns)}")

# 2. Create unified OSM features with state-based IDs
df_osm_unified = df_osm.copy()
df_osm_unified['original_station_id'] = df_osm_unified['station_id']
df_osm_unified['station_id'] = df_osm_unified['unified_station_id']
df_osm_unified = df_osm_unified.drop(['normalized_name', 'unified_station_id'], axis=1)

# Reorder columns: station info first
info_cols = ['station_id', 'original_station_id', 'station_name', 'latitude', 'longitude']
feature_cols = [col for col in df_osm_unified.columns if col not in info_cols]
df_osm_unified = df_osm_unified[info_cols + sorted(feature_cols)]

print(f"\n✓ Created unified OSM features: {len(df_osm_unified)} stations")
print(f"  Columns: {len(df_osm_unified.columns)}")

print("\n" + "=" * 70)
print("SAMPLE OF UNIFIED DATA")
print("=" * 70)
print("\nUnified station_locations (first 5 rows):")
print(df_locations_unified.head())

print("\nUnified OSM features (first 3 rows, selected columns):")
display_cols = ['station_id', 'original_station_id', 'station_name', 'latitude', 'longitude', '_total_elements']
print(df_osm_unified[display_cols].head(3))

Creating unified datasets...
✓ Created unified station_locations: 567 stations
  Columns: ['station_id', 'original_station_id', 'station_name', 'latitude', 'longitude', 'city', 'state']

✓ Created unified OSM features: 230 stations
  Columns: 33

SAMPLE OF UNIFIED DATA

Unified station_locations (first 5 rows):
  station_id original_station_id  \
0      TN005           site_5094   
1      KA012           site_5124   
2      HR026            site_147   
3        NaN           site_5468   
4      DL034            site_119   

                                      station_name   latitude  longitude  \
0                SIDCO Kurichi, Coimbatore - TNPCB  10.942451  76.978996   
1                    Urban, Chamarajanagar - KSPCB  11.553580  76.555210   
2                    MD University, Rohtak - HSPCB  28.521230  76.371380   
3  IESD Banaras Hindu University, Varanasi - UPPCB  25.262326  82.995408   
4                           Sirifort, Delhi - CPCB  28.550425  77.215938   

   city  stat

In [22]:
# Save unified datasets to CSV
print("Saving unified datasets...")
print("=" * 70)

# Save unified station_locations
output_file_locations = 'station_locations_unified.csv'
df_locations_unified.to_csv(output_file_locations, index=False)
print(f"✓ Saved unified station locations to '{output_file_locations}'")
print(f"  - Rows: {len(df_locations_unified)}")
print(f"  - Columns: {len(df_locations_unified.columns)}")

# Save unified OSM features
output_file_osm = 'station_osm_features_unified.csv'
df_osm_unified.to_csv(output_file_osm, index=False)
print(f"\n✓ Saved unified OSM features to '{output_file_osm}'")
print(f"  - Rows: {len(df_osm_unified)}")
print(f"  - Columns: {len(df_osm_unified.columns)}")

print("\n" + "=" * 70)
print("UNIFICATION COMPLETE!")
print("=" * 70)
print("\nKey changes:")
print("  • Station IDs from 'site_XXXX' format mapped to state-based IDs (AP001, DL001, etc.)")
print("  • Original site_XXXX IDs preserved in 'original_station_id' column")
print("  • Stations not found in stations.csv retain their site_XXXX IDs")
print(f"\nUnified datasets can now be merged with stations.csv using 'station_id'!")

# Show ID distribution
print("\n" + "-" * 70)
print("UNIFIED ID DISTRIBUTION")
print("-" * 70)
state_based_ids = df_osm_unified['station_id'].str.match(r'^[A-Z]{2}\d{3}$').sum()
site_based_ids = df_osm_unified['station_id'].str.match(r'^site_\d+$').sum()
print(f"State-based IDs (AP001, DL001, etc.): {state_based_ids}")
print(f"Site-based IDs (site_XXXX): {site_based_ids}")
print(f"Total: {len(df_osm_unified)}")

Saving unified datasets...
✓ Saved unified station locations to 'station_locations_unified.csv'
  - Rows: 567
  - Columns: 7

✓ Saved unified OSM features to 'station_osm_features_unified.csv'
  - Rows: 230
  - Columns: 33

UNIFICATION COMPLETE!

Key changes:
  • Station IDs from 'site_XXXX' format mapped to state-based IDs (AP001, DL001, etc.)
  • Original site_XXXX IDs preserved in 'original_station_id' column
  • Stations not found in stations.csv retain their site_XXXX IDs

Unified datasets can now be merged with stations.csv using 'station_id'!

----------------------------------------------------------------------
UNIFIED ID DISTRIBUTION
----------------------------------------------------------------------
State-based IDs (AP001, DL001, etc.): 191
Site-based IDs (site_XXXX): 0
Total: 230


## Find Unmapped Stations from stations.csv
Identify which stations from stations.csv are NOT present in the unified dataset (not yet processed)

In [27]:
# Find stations in stations.csv that are NOT in unified dataset
print("Analyzing stations from stations.csv not yet in unified dataset...")
print("=" * 70)

# Load the datasets (if not already loaded)
if 'df_stations_master' not in locals():
    df_stations_master = pd.read_csv('stations.csv')
if 'df_osm_unified' not in locals():
    df_osm_unified = pd.read_csv('station_osm_features_unified.csv')

df_osm_unified = pd.read_csv('station_osm_features_unified.csv')
# Get the station IDs from both datasets
stations_csv_ids = set(df_stations_master['StationId'].unique())
unified_ids = set(df_osm_unified['station_id'].unique())

# Find missing IDs (in stations.csv but NOT in unified)
unmapped_ids = stations_csv_ids - unified_ids

print(f"Total stations in stations.csv: {len(stations_csv_ids)}")
print(f"Total stations in unified dataset: {len(unified_ids)}")
print(f"Stations in stations.csv NOT yet in unified dataset: {len(unmapped_ids)}")

if unmapped_ids:
    print("\n" + "=" * 70)
    print(f"UNMAPPED STATIONS ({len(unmapped_ids)} stations)")
    print("=" * 70)
    
    # Get full details of unmapped stations from stations.csv
    unmapped_stations = df_stations_master[df_stations_master['StationId'].isin(unmapped_ids)].copy()
    
    # Sort by StationId
    unmapped_stations = unmapped_stations.sort_values('StationId')
    
    print("\nUnmapped Station Details:")
    print("-" * 70)
    for idx, row in unmapped_stations.iterrows():
        print(f"\nStation ID: {row['StationId']}")
        print(f"  Name: {row['StationName']}")
        print(f"  City: {row['City']}")
        print(f"  State: {row['State']}")
        print(f"  Status: {row['Status']}")
    
    # Create a summary DataFrame
    print("\n" + "=" * 70)
    print("SUMMARY TABLE OF UNMAPPED STATIONS")
    print("=" * 70)
    print(unmapped_stations.to_string(index=False))
    
    # Save to CSV for reference
    output_file = 'unmapped_stations_from_stations_csv.csv'
    unmapped_stations.to_csv(output_file, index=False)
    print(f"\n✓ Saved unmapped stations list to '{output_file}'")
    
    # Analyze by state
    print("\n" + "-" * 70)
    print("UNMAPPED STATIONS BY STATE")
    print("-" * 70)
    state_counts = unmapped_stations['State'].value_counts()
    for state, count in state_counts.items():
        print(f"  {state}: {count} stations")
    
    # Analyze by status
    print("\n" + "-" * 70)
    print("UNMAPPED STATIONS BY STATUS")
    print("-" * 70)
    status_counts = unmapped_stations['Status'].value_counts()
    for status, count in status_counts.items():
        status_label = status if pd.notna(status) and status != '' else 'Unknown/Inactive'
        print(f"  {status_label}: {count} stations")
    
else:
    print("\n✓ All stations from stations.csv are present in the unified dataset!")
    print("  No unmapped stations found.")

Analyzing stations from stations.csv not yet in unified dataset...
Total stations in stations.csv: 230
Total stations in unified dataset: 200
Stations in stations.csv NOT yet in unified dataset: 31

UNMAPPED STATIONS (31 stations)

Unmapped Station Details:
----------------------------------------------------------------------

Station ID: BR008
  Name: Muradpur, Patna - BSPCB
  City: Patna
  State: Bihar
  Status: Active

Station ID: DL006
  Name: Burari Crossing, Delhi - IMD
  City: Delhi
  State: Delhi
  Status: Inactive

Station ID: DL011
  Name: East Arjun Nagar, Delhi - CPCB
  City: Delhi
  State: Delhi
  Status: Active

Station ID: DL021
  Name: NSIT Dwarka, Delhi - CPCB
  City: Delhi
  State: Delhi
  Status: Active

Station ID: HR002
  Name: Arya Nagar, Bahadurgarh - HSPCB
  City: Bahadurgarh
  State: Haryana
  Status: Active

Station ID: HR004
  Name: H.B. Colony, Bhiwani - HSPCB
  City: Bhiwani
  State: Haryana
  Status: Active

Station ID: HR005
  Name: Municipal Corporation

## Analyze Blank Values in station_osm_features_batch_230.csv

In [28]:
# Load the batch file
df_batch = pd.read_csv('station_osm_features_batch_230.csv')

# Calculate blank/missing values
total_cells = df_batch.size  # Total number of cells in the dataframe
blank_values = df_batch.isna().sum().sum()  # Count all NaN/null values
empty_strings = (df_batch == '').sum().sum()  # Count empty strings
total_blank = blank_values + empty_strings

# Calculate statistics
non_blank = total_cells - total_blank
blank_percentage = (total_blank / total_cells) * 100

print(f"File: station_osm_features_batch_230.csv")
print(f"{'='*60}")
print(f"Total rows: {len(df_batch)}")
print(f"Total columns: {len(df_batch.columns)}")
print(f"Total cells: {total_cells:,}")
print(f"\nBlank Value Analysis:")
print(f"  NaN/Null values: {blank_values:,}")
print(f"  Empty strings: {empty_strings:,}")
print(f"  Total blank values: {total_blank:,}")
print(f"  Non-blank values: {non_blank:,}")
print(f"  Blank percentage: {blank_percentage:.2f}%")

# Show blank values per column
print(f"\n{'='*60}")
print("Blank values per column:")
print(f"{'='*60}")
blank_per_column = df_batch.isna().sum() + (df_batch == '').sum()
blank_per_column_sorted = blank_per_column.sort_values(ascending=False)

for col, count in blank_per_column_sorted.items():
    percentage = (count / len(df_batch)) * 100
    print(f"{col:30s}: {count:3d} / {len(df_batch)} ({percentage:5.1f}%)")

File: station_osm_features_batch_230.csv
Total rows: 230
Total columns: 32
Total cells: 7,360

Blank Value Analysis:
  NaN/Null values: 3,374
  Empty strings: 0
  Total blank values: 3,374
  Non-blank values: 3,986
  Blank percentage: 45.84%

Blank values per column:
military                      : 225 / 230 ( 97.8%)
industrial                    : 222 / 230 ( 96.5%)
aeroway                       : 219 / 230 ( 95.2%)
emergency                     : 217 / 230 ( 94.3%)
craft                         : 211 / 230 ( 91.7%)
historic                      : 192 / 230 ( 83.5%)
sport                         : 155 / 230 ( 67.4%)
waterway                      : 147 / 230 ( 63.9%)
water                         : 146 / 230 ( 63.5%)
power                         : 139 / 230 ( 60.4%)
office                        : 135 / 230 ( 58.7%)
public_transport              : 133 / 230 ( 57.8%)
healthcare                    : 133 / 230 ( 57.8%)
man_made                      : 125 / 230 ( 54.3%)
tourism           